In [ ]:
# Google Colab-only setup. No need to run this cell in other environments.
use_colab = False

if use_colab:
    # Mount my Google Drive root folder
    from google.colab import drive
    drive.mount('/content/drive')

    # cd to bayesian-dl-experiments directory
    %cd 'drive/My Drive/Colab Notebooks/bayesian-dl-experiments'
    !ls

## Experiment Setup

### Random seed / PyTorch / CUDA related

In [ ]:
import torch
import numpy as np

# IPython reloading magic
%load_ext autoreload
%autoreload 2

# Random seeds
# Based on https://pytorch.org/docs/stable/notes/randomness.html
torch.manual_seed(682)
np.random.seed(682)

# torch.device / CUDA Setup
use_cuda = True

if use_cuda and torch.cuda.is_available():
    torch_device = torch.device('cuda')
    torch.backends.cudnn.deterministic = True
    # Note: https://discuss.pytorch.org/t/what-does-torch-backends-cudnn-benchmark-do/5936
    torch.backends.cudnn.benchmark = False
    use_pin_memory=True # Faster Host to GPU copies with page-locked memory
else:
    torch_device = torch.device('cpu')
    use_pin_memory=False

### Variable settings

In [ ]:
# Random seed for toy dataset
dataset_toy_random_seed = 691 #@param {type:"integer"}

# Toy dataset size
dataset_toy_size = 10000

# Toy dataset x distribution (uniform) parameters
dataset_toy_x_low = -5
dataset_toy_x_high = 5

# Toy dataset y distribution (normal with noise)
dataset_toy_y_mean = 0
dataset_toy_y_std = 9

# Training set size
dataset_train_size = 0.8 #@param {type:"slider", min:0.1, max:0.9, step:0.05}

# L2 regularization strength
reg_strength = 0.01 #@param {type:"slider", min:0, max:1.0, step:0.05}

# Epochs
n_epochs = 40 #@param {type:"integer"}

# Number of different data splits to try
n_splits = 20 #@param {type:"integer"}

# Data batch sizes
n_training_batch = 128 #@param {type:"integer"}

# Number of test predictions (for each data point)
n_predictions = 10000 #@param {type:"integer"}

## Prepare data

### Get the data as a torch Dataset object

In [ ]:
from torch.utils.data import random_split, DataLoader
from ronald_bdl import datasets

dataset = datasets.ToyDatasets(
    random_seed=dataset_toy_random_seed,
    n_samples=dataset_toy_size,
    x_low=dataset_toy_x_low,
    x_high=dataset_toy_x_high,
    y_mean=dataset_toy_y_mean,
    y_std=dataset_toy_y_std,
)

# Set the training/test set sizes
train_size = int(dataset_train_size * len(dataset))
test_size = len(dataset) - train_size
    
# Print the size of the dataset
print("dataset size = " + str((len(dataset), dataset.n_features)))
print("training set size = " + str(train_size))
print("testing set size = " + str(test_size))

## Define network

In [ ]:
from ronald_bdl import models

network = models.FCNetMCDropout(
    input_dim=dataset.n_features, 
    output_dim=dataset.n_targets,
    hidden_dim=10,
    n_hidden=2,
    dropout_rate=0.01,
)

# Send the whole model to the selected torch.device
network.to(torch_device)

# Print the network structure
print(network)

## Train the network

### Setup

In [ ]:
from torch import nn, optim

# Model to train mode
network.train()

# Mean Squared Error for loss function to minimize
objective = nn.MSELoss()

rmse_non_mc, rmse_mc, test_lls_mc = [], [], []

### Train/test the model

In [ ]:
import time

for s in range(n_splits):

    # Prepare new train-test split
    train, test = random_split(dataset, lengths=[train_size, test_size])
    train_loader = DataLoader(train, batch_size=n_training_batch, pin_memory=use_pin_memory)
    if use_pin_memory: test.dataset.data.pin_memory()
    
    # Adam optimizer
    # https://pytorch.org/docs/stable/optim.html?highlight=adam#torch.optim.Adam
    # NOTE: Need to set L2 regularization from here
    optimizer = optim.Adam(
        network.parameters(),
        lr=0.01,
        weight_decay=reg_strength, # L2 regularization
    )
    
    """
    Training
    """

    print("Starting split " + str(s))

    # Record training start time (for this split)
    tic = time.time()

    for epoch in range(n_epochs): # loop over the dataset multiple times

        for i, data in enumerate(train_loader):
            # get the inputs; data is a list of [inputs, labels]
            inputs, targets = data

            # Store the batch to torch_device's memory
            inputs = inputs.to(torch_device)
            targets = targets.to(torch_device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = network(inputs)

            loss = objective(outputs, targets)
            loss.backward()

            optimizer.step()
            
    # Record training end time
    toc = time.time()

    # Report the final loss
    print("Split " + str(s) + ", final loss = " + str(loss.item()))            

    """
    Testing
    """
    # Model to eval mode
    network.eval()

    # Get the test data
    inputs, targets = test.dataset[test.indices]

    # Store the batch to torch_device's memory
    inputs = inputs.to(torch_device)
    targets = targets.to(torch_device)

    # Record testing start time (for this split)
    tic_testing = time.time()    
    
    predictions, mean, var, metrics = network.mc_predict(inputs, n_predictions,
                                                         y_test=targets, reg_strength=reg_strength)

    # Record testing end time
    toc_testing = time.time()    
    
    """
    Print results
    """
    print()
    print("Running split " + str(s) + " test:")
    print("Mean = " + str(mean))
    print("Variance = " + str(var))

    # Print and store additional metrics
    if len(metrics) > 0:
        for key, value in metrics.items():
            print(str(key) + " = " + str(value))

            if key == 'rmse_mc': rmse_mc.append(value.item())
            elif key == 'rmse_non_mc': rmse_non_mc.append(value.item())
            elif key == 'test_ll_mc': test_lls_mc.append(value.item())

    # Report the total training time
    print("Split " + str(s) + " training time = " + str(toc - tic) + " seconds")
    
    # Report the total testing time
    print("Split " + str(s) + " testing time = " + str(toc_testing - tic_testing) + " seconds")
    print()

### Print statistics

In [ ]:
# Copied from DropoutUncertaintyExps repo
print('non-MC RMSE %f +- %f (stddev) +- %f (std error), median %f 25p %f 75p %f \n' % (
        np.mean(rmse_non_mc), np.std(rmse_non_mc), np.std(rmse_non_mc)/np.sqrt(n_splits),
        np.percentile(rmse_non_mc, 50), np.percentile(rmse_non_mc, 25), np.percentile(rmse_non_mc, 75)))

print('MC RMSE %f +- %f (stddev) +- %f (std error), median %f 25p %f 75p %f \n' % (
        np.mean(rmse_mc), np.std(rmse_mc), np.std(rmse_mc)/np.sqrt(n_splits),
        np.percentile(rmse_mc, 50), np.percentile(rmse_mc, 25), np.percentile(rmse_mc, 75)))

print('MC Test Log-likelihood %f +- %f (stddev) +- %f (std error), median %f 25p %f 75p %f \n' % (
        np.mean(test_lls_mc), np.std(test_lls_mc), np.std(test_lls_mc)/np.sqrt(n_splits), 
        np.percentile(test_lls_mc, 50), np.percentile(test_lls_mc, 25), np.percentile(test_lls_mc, 75)))